# OmniSpatial End-to-End Tutorial

This notebook builds a small Xenium-like dataset, converts it to OME-NGFF and SpatialData, validates the output, and outlines how to explore the results in napari and the web viewer. Run the notebook from the repository root using Poetry to ensure all dependencies are available.

In [ ]:
from __future__ import annotations

import json
import shutil
import subprocess
from pathlib import Path

import numpy as np
import pandas as pd
import tifffile

project_root = Path.cwd().resolve()
examples_dir = project_root / "examples" / "artifacts"
dataset_dir = examples_dir / "xenium_synth"
ngff_path = examples_dir / "xenium_ngff.zarr"
spatial_path = examples_dir / "xenium_sdata.zarr"
report_path = examples_dir / "ngff_report.json"

examples_dir.mkdir(parents=True, exist_ok=True)
shutil.rmtree(dataset_dir, ignore_errors=True)
dataset_dir.mkdir(parents=True, exist_ok=True)


## 1. Generate a synthetic dataset

In [ ]:
(dataset_dir / "images").mkdir(parents=True, exist_ok=True)
image = np.array(
    [
        [0, 10, 20, 30],
        [10, 50, 60, 20],
        [20, 70, 90, 40],
        [5, 15, 25, 35],
    ],
    dtype=np.uint16,
)
tifffile.imwrite(dataset_dir / "images" / "synthetic.tif", image, photometric="minisblack")


In [ ]:
cells = pd.DataFrame(
    {
        "cell_id": ["cell_1", "cell_2"],
        "x": [1.0, 2.5],
        "y": [1.0, 2.0],
        "area": [1.0, 1.5],
        "polygon_wkt": [
            "POLYGON ((0.5 0.5, 1.5 0.5, 1.5 1.5, 0.5 1.5, 0.5 0.5))",
            "POLYGON ((2 1.5, 3 1.5, 3 2.5, 2 2.5, 2 1.5))",
        ],
    }
)
cells.to_csv(dataset_dir / "cells.csv", index=False)
matrix = pd.DataFrame(
    {
        "cell_id": ["cell_1", "cell_1", "cell_2"],
        "gene": ["G1", "G2", "G1"],
        "count": [10, 5, 3],
    }
)
matrix.to_csv(dataset_dir / "matrix.csv", index=False)


## 2. Convert to NGFF and SpatialData

In [ ]:
shutil.rmtree(ngff_path, ignore_errors=True)
shutil.rmtree(spatial_path, ignore_errors=True)

omnispatial_dir = project_root / "omnispatial"
subprocess.run(
    [
        "poetry",
        "run",
        "omnispatial",
        "convert",
        str(dataset_dir),
        "--out",
        str(ngff_path),
        "--format",
        "ngff",
        "--vendor",
        "xenium",
    ],
    cwd=omnispatial_dir,
    check=True,
)
subprocess.run(
    [
        "poetry",
        "run",
        "omnispatial",
        "convert",
        str(dataset_dir),
        "--out",
        str(spatial_path),
        "--format",
        "spatialdata",
        "--vendor",
        "xenium",
    ],
    cwd=omnispatial_dir,
    check=True,
)


## 3. Validate and inspect the report

In [ ]:
if report_path.exists():
    report_path.unlink()
subprocess.run(
    [
        "poetry",
        "run",
        "omnispatial",
        "validate",
        str(ngff_path),
        "--format",
        "ngff",
        "--json",
        str(report_path),
    ],
    cwd=omnispatial_dir,
    check=True,
)
report = json.loads(report_path.read_text())
print(json.dumps(report, indent=2))


## 4. Explore the results

- Launch napari with the NGFF bundle and enable the "OmniSpatial Inspector" dock widget:
  ```bash
  cd omnispatial
  poetry run napari ../examples/artifacts/xenium_ngff.zarr
  ```
- Serve the NGFF Zarr over HTTP and open the web viewer:
  ```bash
  cd examples/artifacts
  python -m http.server 8000
  # In another terminal
  cd viewer
  pnpm dev
  ```
  Visit http://localhost:3000 and load `http://localhost:8000/xenium_ngff.zarr`.